# CLAUDE RAG Evaluation
Modified Notebook Authored by:

- [Anthony Gasbarro](https://github.com/AwkwaBear/RAG-model-ee693b)
- Chris Aguilar
- Maxwell Pauly
- (_Original by: [Aymeric Roucher](https://huggingface.co/m-ric)_)

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.


RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system's performance!
So let's see how to evaluate our RAG system.

### Evaluating RAG performance

Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.

For our evaluation pipeline, we will need:
1. An evaluation dataset with question - answer couples (QA couples)
2. An evaluator to compute the accuracy of our system on the above evaluation dataset.

➡️ It turns out, we can use LLMs to help us all along the way!
1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

# Initial Conda Environment Setup
- In command line:
    - `conda create -n <PICK SOME ENVIRONMENT NAME>`
    - `conda install python=3.12 pytorch pytorch-cuda transformers accelerate sentence-transformers faiss-gpu openpyxl python-dotenv -c pytorch -c nvidia -c conda-forge -y`
- Create open this notebook in VScode and set the jupyter interpreter to `<PICK SOME ENVIRONMENT NAME>`

## Run the pip install below for the rest of the required packages


In [1]:
!pip install -q torch transformers transformers langchain langchain-anthropic sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille ipywidgets jupyter plotly

In [2]:
%reload_ext autoreload
%autoreload 2

Imports

In [3]:
from dotenv import load_dotenv
import os
from tqdm.auto import tqdm # importing a library for progress bars
import pandas as pd # pandas is a library for data manipulation
from typing import Optional, List, Tuple # importing some type hints
import json # importing a library for working with json
import datasets # importing the datasets library
from huggingface_hub import notebook_login 
from datasets import load_dataset
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from huggingface_hub import InferenceClient
import random
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.llms import HuggingFaceHub
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from langchain_core.language_models import BaseChatModel
from langchain.chat_models import ChatOpenAI
import glob
import plotly.express as px

## Load Environment variables from .env file
- Loads API keys etc. 

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
print("Will print true if the .env file is loaded, false if not")
load_dotenv()


# print(f"Hugging Face API Token: {os.getenv("HF_API_TOKEN")} ")
# print(f"OpenAI API Key: {os.getenv("OPENAI_API_KEY")} ")

Will print true if the .env file is loaded, false if not


True

In [5]:
import pandas as pd # pandas is a library for data manipulation
from typing import Optional, List, Tuple # importing some type hints
import json # importing a library for working with json
import datasets # importing the datasets library

pd.set_option("display.max_colwidth", None) # setting the maximum column width for pandas

## Download Hugging Face Doc Dataset

In [6]:
import os
import json
from datasets import load_dataset
from huggingface_hub import login

# File path for the dataset
path_hf_doc_dataset = "./hf_docs_dataset/huggingface_doc_dataset.json"


def recreate_dataset():
    # Log in to the Hugging Face Hub
    hf_api_token = os.getenv("HF_API_TOKEN")
    if not hf_api_token:
        raise EnvironmentError("HF_API_TOKEN is not set in the environment.")
    login(hf_api_token)

    # Load the dataset from Hugging Face
    ds = load_dataset("m-ric/huggingface_doc", split="train")

    # Convert the dataset to a list of dictionaries
    data_list = [doc for doc in ds]

    # Ensure the directory exists
    os.makedirs(os.path.dirname(path_hf_doc_dataset), exist_ok=True)

    # Write the dataset content to a JSON file
    with open(path_hf_doc_dataset, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

    print(f"Dataset saved to {path_hf_doc_dataset}")
    return data_list



# Check if the dataset file exists
if os.path.exists(path_hf_doc_dataset) and os.path.getsize(path_hf_doc_dataset) > 0:
    try:
        with open(path_hf_doc_dataset, "r", encoding="utf-8") as f:
            data_list = json.load(f)
    except (json.JSONDecodeError, OSError) as e:
        print(f"Error loading JSON file: {e}. Recreating dataset.")
        data_list = recreate_dataset()
else:
    data_list = recreate_dataset()


# 1. Build a synthetic dataset for evaluation
We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.

Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

### 1.1. Prepare source documents

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(data_list)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

### 1.2. Setup agents for question generation

We use [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation because it it has excellent performance in leaderboards such as [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [8]:
from huggingface_hub import InferenceClient
import logging
import json


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

def call_llm(inference_client, prompt):
    try:
        # Make the API call
        response = inference_client.post(
            json={
                "inputs": prompt,
                "parameters": {"max_new_tokens": 1000},
                "task": "text-generation",
            },
        )

        # Decode the raw response bytes to a string
        response_str = response.decode("utf-8")

        # Parse and validate the response JSON
        response_data = json.loads(response_str)
        if not isinstance(response_data, list) or not response_data or "generated_text" not in response_data[0]:
            logging.error(f"Unexpected response format: {response_data}")
            raise ValueError(f"Invalid response format: {response_data}")

        # Return the generated text
        return response_data[0]["generated_text"]

    except json.JSONDecodeError as json_err:
        logging.error(f"JSON decoding error: {json_err}. Response: {response}")
        raise
    except AttributeError as attr_err:
        logging.error(f"Attribute error: {attr_err}. Ensure `response` is correctly handled.")
        raise
    except Exception as e:
        logging.error(f"Unexpected error in call_llm: {e}")
        raise


# def call_llm(inference_client: InferenceClient, prompt: str):
#     response = inference_client.post(
#         json={
#             "inputs": prompt,
#             "parameters": {"max_new_tokens": 1000},
#             "task": "text-generation",
#         },
#     )
#     return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [9]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

Now let's generate our QA couples.
For this example, we generate only 10 QA couples and will load the rest from the Hub.

But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.

In [10]:
import random

N_GENERATIONS = 500 # We intentionally generate only 10 QA couples here for cost and time considerations

# Save outputs to a JSON file
QA_initial_couples_path = './generated_QAs/initial_generated_qa_couples.json'


def generate_initial_QA():
    print(f"Generating {N_GENERATIONS} QA couples...")

    QA_initial_outputs = []
    for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
        try:
            # Generate QA couple
            output_QA_couple = call_llm(
                llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
            )
            question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
            answer = output_QA_couple.split("Answer: ")[-1]
            assert len(answer) < 500, f"Answer is too long: {answer}"
            QA_initial_outputs.append(
                {
                    "context": sampled_context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": sampled_context.metadata["source"],
                }
            )
        except AssertionError as ae:
            logging.error(f"AssertionError: {ae}. Context: {sampled_context.page_content}")
        except ValueError as ve:
            logging.error(f"ValueError in call_llm: {ve}")
        except Exception as e:
            logging.error(f"Unexpected error during loop iteration: {e}. Context: {sampled_context.page_content}")
        
    # Ensure the directory exists
    os.makedirs(os.path.dirname(QA_initial_couples_path), exist_ok=True)

    with open(QA_initial_couples_path, 'w', encoding='utf-8') as json_file:
        json.dump(QA_initial_outputs, json_file, ensure_ascii=False, indent=4)

    print(f"Generated QA couples saved to {QA_initial_couples_path}")
        
    return QA_initial_outputs



# Check if the dataset file exists
if os.path.exists(QA_initial_couples_path) and os.path.getsize(QA_initial_couples_path) > 0:
    try:
        with open(QA_initial_couples_path, "r", encoding="utf-8") as f:
            QA_initial_outputs = json.load(f)
            print(f"Successfully LOADED {len(QA_initial_outputs)} QA couples")
    except (json.JSONDecodeError, OSError) as e:
        print(f"Error loading JSON file: {e}. Recreating dataset.")
        QA_initial_outputs = generate_initial_QA()
else:
    QA_initial_outputs = generate_initial_QA()
    



# Display the first QA couple to check the format
print(f"Below is the first QA couple generated:")
display(pd.DataFrame(QA_initial_outputs).head(1))


Generating 500 QA couples...


  0%|          | 0/500 [00:00<?, ?it/s]

ERROR:root:AssertionError: Answer is too long: You can replace the DDIMScheduler with the DDPMScheduler in the Kandinsky text-to-image pipeline by creating a new instance of the DDPMScheduler and passing it to the pipeline's scheduler attribute. Here's an example:

```
from diffusers import DDPMScheduler
from diffusers import DiffusionPipeline

# Create a new instance of the DDPMScheduler
scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

# Pass the new scheduler to the pipeline's scheduler attribute
pipe.scheduler = scheduler
```. Context: <div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/robot_cat.png"/>
</div>

## Optimizations

Kandinsky is unique because it requires a prior pipeline to generate the mappings, and a second pipeline to decode the latents into an image. Optimization efforts should be focused on the second pipeline because that is where the bulk of

Generated QA couples saved to ./generated_QAs/initial_generated_qa_couples.json
Below is the first QA couple generated:


,context,question,answer,source_doc
0,### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\n- Allow users to submit with enter in Interfaces with textbox / number inputs [@aliabid94](https://github.com/aliabid94) in [PR 4090](https://github.com/gradio-app/gradio/pull/4090).\n- Updates gradio's requirements.txt to requires uvicorn>=0.14.0 by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Updates some error messaging by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Renames simplified Chinese translation file from `zh-cn.json` to `zh-CN.json` by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.3\n\n### New Features:\n\nNo changes to highlight.\n\n### Bug Fixes:\n\n- Fixes issue with indentation in `gr.Code()` component with streaming by [@dawoodkhan82](https://github.com/dawoodkhan82) in [PR 4043](https://github.com/gradio-app/gradio/pull/4043)\n\n### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\nNo changes to highlight.\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.2\n\n### Bug Fixes\n\n- Code component visual updates by [@pngwn](https://github.com/pngwn) in [PR 4051](https://github.com/gradio-app/gradio/pull/4051)\n\n### New Features:,What was fixed in the Code component in version 3.28.2?\n,The Code component had visual updates in version 3.28.2.,gradio-app/gradio/blob/main/CHANGELOG.md


### 1.3. Setup critique agents

The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practicioners.

One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___

We now build and run these critique agents.

In [11]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [12]:
import re # importing the regex library

# Save outputs to a JSON file
QA_critiqued_couples_path = './generated_QAs/critiqued_qa_couples.json'

# This is a helper function to parse the evaluation response
def parse_evaluation(evaluation):
    try:
        # Use regex to extract 'Evaluation' and 'Total rating'
        eval_match = re.search(r"Evaluation:\s*(.+)", evaluation, re.DOTALL)
        score_match = re.search(r"Total rating:\s*(\d+)", evaluation)
        
        # Ensure both matches were found
        if not eval_match or not score_match:
            raise ValueError("Missing 'Evaluation' or 'Total rating' in response")
        
        eval_text = eval_match.group(1).strip()
        score = int(score_match.group(1))
        return score, eval_text
    except Exception as e:
        raise ValueError(f"Error parsing evaluation: {e}")

# This is the main function to generate critiques for each QA couple
def generate_critiqued_QA(QA_initial_outputs):
    
    # Create a new list to store the critiqued QA couples 
    # initialize it with the initial QA couples
    QA_critiqued_outputs = QA_initial_outputs
    
    print("Generating critique for each QA couple...")
    
    for output in tqdm(QA_critiqued_outputs):
        try:
            evaluations = {
                "groundedness": call_llm(
                    llm_client,
                    question_groundedness_critique_prompt.format(
                        context=output["context"], question=output["question"]
                    ),
                ),
                "relevance": call_llm(
                    llm_client,
                    question_relevance_critique_prompt.format(question=output["question"]),
                ),
                "standalone": call_llm(
                    llm_client,
                    question_standalone_critique_prompt.format(question=output["question"]),
                ),
            }
            for criterion, evaluation in evaluations.items():
                try:
                    score, eval_text = parse_evaluation(evaluation)
                    output.update(
                        {
                            f"{criterion}_score": score,
                            f"{criterion}_eval": eval_text,
                        }
                    )
                except ValueError as ve:
                    print(f"Error parsing score or evaluation for criterion '{criterion}': {ve}")
        except Exception as e:
                print(f"Unexpected error for output {output}: {e}")
                continue
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(QA_critiqued_couples_path), exist_ok=True)

    with open(QA_critiqued_couples_path, 'w', encoding='utf-8') as json_file:
        json.dump(QA_critiqued_outputs, json_file, ensure_ascii=False, indent=4)

    print(f"Generated QA couples saved to {QA_critiqued_couples_path}")

    return QA_critiqued_outputs


# Check if the dataset file exists
if os.path.exists(QA_critiqued_couples_path) and os.path.getsize(QA_critiqued_couples_path) > 0:
    try:
        with open(QA_critiqued_couples_path, "r", encoding="utf-8") as f:
            QA_critiqued_outputs = json.load(f)
            print(f"Successfully LOADED {len(QA_critiqued_outputs)} QA Critiques")
    except (json.JSONDecodeError, OSError) as e:
        print(f"Error loading JSON file: {e}. Recreating dataset.")
        QA_critiqued_outputs = generate_critiqued_QA(QA_initial_outputs)
else:
    QA_critiqued_outputs = generate_critiqued_QA(QA_initial_outputs)
    



# Display the first QA couple to check the format
print(f"Below is the CRITIQUED QA couple:")
display(pd.DataFrame(QA_critiqued_outputs).head(1))
    



Generating critique for each QA couple...


  0%|          | 0/474 [00:00<?, ?it/s]

Error parsing score or evaluation for criterion 'standalone': Error parsing evaluation: Missing 'Evaluation' or 'Total rating' in response
Error parsing score or evaluation for criterion 'groundedness': Error parsing evaluation: Missing 'Evaluation' or 'Total rating' in response
Error parsing score or evaluation for criterion 'standalone': Error parsing evaluation: Missing 'Evaluation' or 'Total rating' in response
Generated QA couples saved to ./generated_QAs/critiqued_qa_couples.json
Below is the CRITIQUED QA couple:


,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
0,### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\n- Allow users to submit with enter in Interfaces with textbox / number inputs [@aliabid94](https://github.com/aliabid94) in [PR 4090](https://github.com/gradio-app/gradio/pull/4090).\n- Updates gradio's requirements.txt to requires uvicorn>=0.14.0 by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Updates some error messaging by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Renames simplified Chinese translation file from `zh-cn.json` to `zh-CN.json` by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.3\n\n### New Features:\n\nNo changes to highlight.\n\n### Bug Fixes:\n\n- Fixes issue with indentation in `gr.Code()` component with streaming by [@dawoodkhan82](https://github.com/dawoodkhan82) in [PR 4043](https://github.com/gradio-app/gradio/pull/4043)\n\n### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\nNo changes to highlight.\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.2\n\n### Bug Fixes\n\n- Code component visual updates by [@pngwn](https://github.com/pngwn) in [PR 4051](https://github.com/gradio-app/gradio/pull/4051)\n\n### New Features:,What was fixed in the Code component in version 3.28.2?\n,The Code component had visual updates in version 3.28.2.,gradio-app/gradio/blob/main/CHANGELOG.md,3.0,"(your rationale for the rating, as a text)\nTotal rating: (your rating, as a number between 1 and 5)\n\nYou MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.\n\nNow here are the question and context.\n\nQuestion: What was fixed in the Code component in version 3.28.2?\n\n\nContext: ### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\n- Allow users to submit with enter in Interfaces with textbox / number inputs [@aliabid94](https://github.com/aliabid94) in [PR 4090](https://github.com/gradio-app/gradio/pull/4090).\n- Updates gradio's requirements.txt to requires uvicorn>=0.14.0 by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Updates some error messaging by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n- Renames simplified Chinese translation file from `zh-cn.json` to `zh-CN.json` by [@abidlabs](https://github.com/abidlabs) in [PR 4086](https://github.com/gradio-app/gradio/pull/4086)\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.3\n\n### New Features:\n\nNo changes to highlight.\n\n### Bug Fixes:\n\n- Fixes issue with indentation in `gr.Code()` component with streaming by [@dawoodkhan82](https://github.com/dawoodkhan82) in [PR 4043](https://github.com/gradio-app/gradio/pull/4043)\n\n### Documentation Changes:\n\nNo changes to highlight.\n\n### Testing and Infrastructure Changes:\n\nNo changes to highlight.\n\n### Breaking Changes:\n\nNo changes to highlight.\n\n### Full Changelog:\n\nNo changes to highlight.\n\n### Contributors Shoutout:\n\nNo changes to highlight.\n\n## 3.28.2\n\n### Bug Fixes\n\n- Code component visual updates by [@pngwn](https://github.com/pngwn) in [PR 4051](https://github.com/gradio-app/gradio/pull/4051)\n\n### New Features:\n\nAnswer::: \nThe Code compo

Now let us filter out bad questions based on our critique agent scores:

In [13]:
import os
import json
import pandas as pd
import datasets

pd.set_option("display.max_colwidth", None)

# Save outputs to a JSON file
filtered_QA_couples_path = './generated_QAs/filtered_qa_couples.json'

# this is the main function of actually filtering the QA couples
# any scores that are less than a 4 in each category are filtered out
def filter_questions(dataframe_QA_critiqued_outputs):
    # Ensure input is a DataFrame
    if not isinstance(dataframe_QA_critiqued_outputs, pd.DataFrame):
        filtered_QA_questions = pd.DataFrame.from_dict(dataframe_QA_critiqued_outputs)
    else:
        filtered_QA_questions = dataframe_QA_critiqued_outputs.copy()

    print("Evaluation dataset before filtering:")
    display(
        filtered_QA_questions[
            [
                "question",
                "answer",
                "groundedness_score",
                "relevance_score",
                "standalone_score",
            ]
        ]
    )
    
    # Apply filtering
    filtered_QA_questions = filtered_QA_questions.loc[
        (filtered_QA_questions["groundedness_score"] >= 4)
        & (filtered_QA_questions["relevance_score"] >= 4)
        & (filtered_QA_questions["standalone_score"] >= 4)
    ]

    # Ensure the target directory path exists
    os.makedirs(os.path.dirname(filtered_QA_couples_path), exist_ok=True)
 
    # Write the filtered QA couples to a JSON file
    # Convert to list of dictionaries to ensure JSON serialization
    filtered_QA_questions_json = filtered_QA_questions.to_dict(orient='records')
    
    with open(filtered_QA_couples_path, 'w', encoding='utf-8') as json_file:
        json.dump(filtered_QA_questions_json, json_file, ensure_ascii=False, indent=4)

    print(f"Generated QA couples saved to {filtered_QA_couples_path}")

    return filtered_QA_questions

# Check if file exists and has content
if os.path.exists(filtered_QA_couples_path) and os.path.getsize(filtered_QA_couples_path) > 0:
    try:
        # Load JSON and convert directly to DataFrame
        with open(filtered_QA_couples_path, "r", encoding="utf-8") as f:
            filtered_QA_questions = pd.DataFrame(json.load(f))
            print(f"Successfully LOADED {len(filtered_QA_questions)} Filtered and Finalized QA couples")
    except (json.JSONDecodeError, OSError) as e:
        print(f"Error loading JSON file: {e}. Running QA Filtering on critiqued outputs.")
        filtered_QA_questions = filter_questions(QA_critiqued_outputs)
else:
    filtered_QA_questions = filter_questions(QA_critiqued_outputs)

# Create dataset
eval_dataset = datasets.Dataset.from_pandas(
    filtered_QA_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What was fixed in the Code component in version 3.28.2?\n,The Code component had visual updates in version 3.28.2.,3.0,1,2.0
1,What is the function name to delete a collection item in Hugging Face Hub API?\n,The function name to delete a collection item in Hugging Face Hub API is `delete_collection_item`.,5.0,4,5.0
2,How can I run a pipeline on a large model using 🤗 `accelerate`?\n,"First, make sure you have installed `accelerate` with `pip install accelerate`. Then, load your model using `device_map=""auto""` and pass the `torch_dtype=torch.bfloat16` argument to the `pipeline` function. You can also pass 8-bit loaded models if you install `bitsandbytes` and add the argument `load_in_8bit=True` to the `model_kwargs` parameter.",5.0,5,5.0
3,How do I scroll to the output in the Gradio demo?\n,"To scroll to the output in the Gradio demo, click the ""Scroll"" button.",5.0,4,5.0
4,What is the name of the variant of ResNet called TResNet?\n,TResNet,1.0,1,5.0
...,...,...,...,...,...
469,What is the easiest way to contribute to the Diffusers project?\n,The easiest way to contribute to the Diffusers project is by asking and answering questions on the Diffusers discussion forum or on Discord.,5.0,5,5.0
470,How is the pricing for Inference Endpoints determined?\n,"The pricing for Inference Endpoints is based on hourly compute and billed monthly, with the cost per CPU core/hr being as low as $0.06 and the cost per GPU/hr being as low as $0.6, depending on the user's needs.",5.0,4,5.0
471,What was recently updated in the GooAQ dataset?\n,The GooAQ dataset was recently updated after splits were added for the same. This update includes new train/val/test splits and an updated README.\n```,5.0,2,5.0
472,What is the objective of contrastive learning in vision-language models?\n,The objective of contrastive learning in vision-language models is to map input images and texts to the same feature space such that the distance between the embeddings of image-text pairs is minimized if they match or maximized if they don’t.,5.0,5,5.0


Generated QA couples saved to ./generated_QAs/filtered_qa_couples.json


In [14]:
# Here we print the final evaluation dataset

print("============================================")
print("Final evaluation dataset:")
display(
    filtered_QA_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

display(len(filtered_QA_questions))

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
1,What is the function name to delete a collection item in Hugging Face Hub API?\n,The function name to delete a collection item in Hugging Face Hub API is `delete_collection_item`.,5.0,4,5.0
2,How can I run a pipeline on a large model using 🤗 `accelerate`?\n,"First, make sure you have installed `accelerate` with `pip install accelerate`. Then, load your model using `device_map=""auto""` and pass the `torch_dtype=torch.bfloat16` argument to the `pipeline` function. You can also pass 8-bit loaded models if you install `bitsandbytes` and add the argument `load_in_8bit=True` to the `model_kwargs` parameter.",5.0,5,5.0
3,How do I scroll to the output in the Gradio demo?\n,"To scroll to the output in the Gradio demo, click the ""Scroll"" button.",5.0,4,5.0
9,What is the loss function used in the contrastive training method?\n,"The loss function used in the contrastive training method is given by the formula: -1/n * Σ^n (exp(sim(a_i, p_i)) / Σ^n exp(sim(a_i, p_j))), where n is the number of training samples, a_i and p_i are the sentences from the pair, and sim is the similarity function.",5.0,5,5.0
12,What is the goal of reading BigBird's paper?\n,"The goal of reading BigBird's paper is to extract the necessary information required to effectively re-implement the model in 🤗 Transformers, rather than getting a deep theoretical understanding of the paper.",5.0,5,5.0
...,...,...,...,...,...
464,What is the goal of red-teaming language models?\n,The goal of red-teaming language models is to craft a prompt that would trigger the model to generate text that is likely to cause harm.,5.0,5,5.0
465,What are some tasks that the models of Transformers can perform?\n,"The models of Transformers can perform tasks such as text classification, information extraction, question answering, text summarization, translation, text generation, image classification, object detection, image segmentation, speech recognition, and audio classification. They can also perform tasks in multiple combined modalities, such as optical character recognition, information extraction from scanned documents, video classification, and visual question answering.",5.0,5,5.0
469,What is the easiest way to contribute to the Diffusers project?\n,The easiest way to contribute to the Diffusers project is by asking and answering questions on the Diffusers discussion forum or on Discord.,5.0,5,5.0
470,How is the pricing for Inference Endpoints determined?\n,"The pricing for Inference Endpoints is based on hourly compute and billed monthly, with the cost per CPU core/hr being as low as $0.06 and the cost per GPU/hr being as low as $0.6, depending on the user's needs.",5.0,4,5.0


149

Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

We have generated only a few QA couples here to reduce time and cost. But let's kick start the next part by loading a pre-generated dataset:

# 2. Build our RAG System

### 2.1. Preprocessing documents to build our vector database

- In this part, __we split the documents from our knowledge base into smaller chunks__: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

Many options exist for text splitting:
- split every `n` words / characters, but this has the risk of cutting in half paragraphs or even sentences
- split after `n` words / character, but only on sentence boundaries
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.

[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

> In the following, we use Langchain's `RecursiveCharacterTextSplitter`.

💡 _To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better._

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(data_list)
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### 2.2. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [ ]:
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            allow_dangerous_deserialization=True,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

# MAKE SURE YOU CREATE A HUGGING FACE API KEY AND ADD IT BELOW IN `HF_API_TOKEN = ""`

In [ ]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"
HF_API_TOKEN = os.getenv("HF_API_TOKEN")

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    huggingfacehub_api_token=HF_API_TOKEN,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM


def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

# 3. Benchmarking the RAG system

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evlauation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on faithfulness since it the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [ ]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

# MAKE SURE YOU CREATE A HUGGING FACE API KEY AND ADD IT BELOW IN `OPEN_AI_KEY = ""`

- Other chat models can be loaded following the documentation for [langchain here](https://python.langchain.com/v0.1/docs/modules/model_io/chat/quick_start/)

In [ ]:
from langchain_anthropic import ChatAnthropic

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

eval_chat_model = ChatAnthropic(model="claude-3-sonnet-20240229", api_key=ANTHROPIC_API_KEY)
evaluator_name = "Claude-3-Sonnet"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 Let's run the tests and evaluate answers!👇

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"
            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

### Inspect results

In [ ]:
import glob

reader_judge_outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    reader_judge_outputs.append(output)
result = pd.concat(reader_judge_outputs)

In [ ]:
# Remove rows with N/A values before processing
result = result[result["eval_score_Claude-3-Sonnet"].str.strip().str.lower() != "n/a"]

# Filter out non-integer values between 1-5
result = result[result["eval_score_Claude-3-Sonnet"].str.contains(r'^[1-5]$')]

# Convert to integer
result["eval_score_Claude-3-Sonnet"] = result["eval_score_Claude-3-Sonnet"].astype(int)

result["eval_score_Claude-3-Sonnet"] = (result["eval_score_Claude-3-Sonnet"] - 1) / 4


In [ ]:
average_scores = result.groupby("settings")["eval_score_Claude-3-Sonnet"].mean()
average_scores.sort_values()

Visualizing the results

In [ ]:
average_scores = result.groupby("settings")[["eval_score_Claude-3-Sonnet"]].mean().reset_index()
print(average_scores.columns)
print(average_scores.head())


In [ ]:
import plotly.express as px

# Function to format the settings
def format_settings(settings):
    formatted = settings.replace("chunk:", "Chunk Size: ") \
                        .replace("embeddings:", "Embeddings: ") \
                        .replace("rerank:", "Rerank: ") \
                        .replace("reader-model:", "Reader Model: ") \
                        .replace("evaluator-model:", "Evaluator Model: ") \
                        .replace("~", "/")  # Replace any special characters as needed
    return formatted.replace("_", "<br>")  # Replace underscores with line breaks for better formatting

# Assuming 'average_scores' is a DataFrame that contains 'settings' and 'eval_score_GPT4'
# Create a new column with formatted settings
average_scores['formatted_settings'] = average_scores['settings'].apply(format_settings)

# Multiply the scores by 100 to convert to percentage
average_scores['eval_score_percentage'] = average_scores['eval_score_Claude-3-Sonnet'] * 100

# Now use 'formatted_settings' in the plot
fig = px.bar(
    average_scores,
    x='formatted_settings',  # Use the formatted settings for x-axis
    y='eval_score_percentage',     # Use the percentage scores for y-axis
    labels={
        "eval_score_percentage": "Accuracy",  # Y-axis label
        "formatted_settings": "Configuration",  # X-axis label
    },
    color='eval_score_percentage',  # Color based on the evaluation score
    color_continuous_scale="bluered",  # Color scale
)

# Update layout settings
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],  # Adjusting range to [0, 100] for percentage
    title="<b>Accuracy of Different RAG Configurations for 70 Filtered Q/A Pairs evaluated by claude-3-sonnet-20240229</b>",
    xaxis_title="RAG Settings",
    font=dict(size=11),
)

# Add percentage suffix to the y-axis
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)  # Hide the color scale
fig.update_traces(texttemplate="%{y:.1f}%", textposition="outside")  # Display percentage on bars

# Show the figure
fig.show()

## Example results

Let us load the results that I obtained by tweaking the different options available in this notebook.
For more detail on why these options could work on not, see the notebook on [advanced_RAG](advanced_rag).

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ ___There is no single good recipe: you should try several different directions when tuning your RAG systems.___


In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()



As you can see, these had varying impact on performance. In particular, tuning the chunk size is both easy and very impactful.

But this is our case: your results could be very different: now that you have a robust evaluation pipeline, you can set on to explore other options! 🗺️